# Weekly Sales Performance Report

**Objective:** To analyze weekly sales data by product category from the Snowflake `SALES` table and visualize the top-performing categories.

In [ ]:
import pandas as pd
import snowflake.connector
import os
import matplotlib.pyplot as plt

# Assume Snowflake credentials are set as environment variables for security
SNOWFLAKE_USER = os.getenv('SNOWFLAKE_USER')
SNOWFLAKE_PASSWORD = os.getenv('SNOWFLAKE_PASSWORD')
SNOWFLAKE_ACCOUNT = os.getenv('SNOWFLAKE_ACCOUNT')
SNOWFLAKE_WAREHOUSE = 'COMPUTE_WH'
SNOWFLAKE_DATABASE = 'RETAIL_PROD'
SNOWFLAKE_SCHEMA = 'SALES'

In [ ]:
# [NOTEBOOK_AGENT]: SQL in this cell was automatically translated.
# This cell connects to Snowflake and fetches aggregated sales data.
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
)

sql_query = """
```sql
SELECT
    PRODUCT_CATEGORY,
    SUM(SALE_AMOUNT) as TOTAL_REVENUE
FROM
    `WEEKLY_SALES`
WHERE
    SALE_DATE >= DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY)
GROUP BY
    PRODUCT_CATEGORY
ORDER BY
    TOTAL_REVENUE DESC
LIMIT 10
```
"""

print("Executing query against Snowflake...")
sales_df = pd.read_sql(sql_query, conn)
conn.close()

print("Data loaded successfully.")

In [ ]:
# Perform a quick data check
print("Top 5 Product Categories by Revenue:")
display(sales_df.head())

In [ ]:
# Visualize the results
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(figsize=(12, 7))

ax.bar(sales_df['PRODUCT_CATEGORY'], sales_df['TOTAL_REVENUE'], color='skyblue')
ax.set_title('Top 10 Product Categories by Revenue (Last 7 Days)', fontsize=16)
ax.set_ylabel('Total Revenue ($)')
ax.set_xlabel('Product Category')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()